<a href="https://colab.research.google.com/github/myyiliame/rp/blob/master/shimingbei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.13-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.13-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [3]:
!ls drive/app/desktop/train

train_Application.csv	       train_History_Payment.csv  train_Personas.csv
train_History_Application.csv  train_label.csv


In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from collections import defaultdict
import h5py
import time
import os
import sys
import re
import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from copy import deepcopy
import pprint
import math
import pandas as pd
import numpy as np
from pylab import *

from matplotlib.font_manager import _rebuild
_rebuild()
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False  # 用来正常显示负号
mpl.rcParams['font.sans-serif'] = ['SimHei']
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)
path="drive/app/desktop/"


# 数据读取

训练集

In [0]:
#-----------------------------------------------------------数据读取
#该表为数据主表，记录了每笔贷款申请(以“申请编号”为标识)对应的申请信 息，训练集样本的真实 label 存于数据表 train_label.csv 中
train_Application = pd.read_csv(path + 'train/train_Application.csv')
col_app = list(train_Application.columns)

#该表记录了贷款申请人的个人资产、家庭、社交圈等基本信息
train_Personas = pd.read_csv(path + 'train/train_Personas.csv')
col_per = list(train_Personas)

#该表记录了贷款申请人的历史申请信息，同一个“申 请编号”可对应多笔“历史申请编号”
train_History_Application = pd.read_csv(path + 'train/train_History_Application.csv')
col_his_app = list(train_History_Application.columns)

#该表记录了申请人历史贷款申请的还款信息
train_History_Payment = pd.read_csv(path + 'train/train_History_Payment.csv')
col_his_pay = list(train_History_Payment.columns)

#结果集
train_label = pd.read_csv(path + 'train/train_label.csv')


测试集

In [0]:

##2.应用集
A_Application = pd.read_csv(path + 'A/A_Application.csv')

#该表记录了贷款申请人的个人资产、家庭、社交圈等基本信息
A_Personas = pd.read_csv(path + 'A/A_Personas.csv')

#该表记录了贷款申请人的历史申请信息，同一个“申 请编号”可对应多笔“历史申请编号”
A_History_Application = pd.read_csv(path + 'A/A_History_Application.csv')

#该表记录了申请人历史贷款申请的还款信息
A_History_Payment = pd.read_csv(path + 'A/A_History_Payment.csv')


# 预处理

## 缺失值填充\数据类型转换

In [0]:

#去除利率1、利率2
#---------------------------------------------------------缺失值填充\数据类型转换
def get_data_type(sheet_name):
    type_dict=pd.read_excel(path+'数据字典1.xlsx', sheet_name=sheet_name)
    var_numeric = list(type_dict[type_dict['变量类型']==1]['名称'])
    var_attribute = list(type_dict[type_dict['变量类型']==2]['名称'])
    datatype = dict(zip(type_dict['名称'], type_dict['数据类型']))
    return var_attribute, var_numeric, datatype

##训练集
#application表
app_var_attribute, app_var_numeric, app_datatype = get_data_type('Application')
train_Application.loc[:,app_var_numeric] = train_Application.loc[:,app_var_numeric].fillna(0)
train_Application.loc[:,app_var_attribute] = train_Application.loc[:,app_var_attribute].fillna(-1)
for i in col_app:
    train_Application[i] = train_Application[i].astype(app_datatype[i])
    
A_Application.loc[:,app_var_numeric] = A_Application.loc[:,app_var_numeric].fillna(0)
A_Application.loc[:,app_var_attribute] = A_Application.loc[:,app_var_attribute].fillna(-1)
for i in col_app:
    A_Application[i] = A_Application[i].astype(app_datatype[i])
   

#Personas表
per_var_attribute, per_var_numeric, per_datatype = get_data_type('Personas')
train_Personas.loc[:,per_var_numeric] = train_Personas.loc[:,per_var_numeric].fillna(0)
train_Personas.loc[:,per_var_attribute] = train_Personas.loc[:,per_var_attribute].fillna(-1)
for i in col_per:
    train_Personas[i] = train_Personas[i].astype(per_datatype[i])

A_Personas.loc[:,per_var_numeric] = A_Personas.loc[:,per_var_numeric].fillna(0)
A_Personas.loc[:,per_var_attribute] = A_Personas.loc[:,per_var_attribute].fillna(-1)
for i in col_per:
    A_Personas[i] = A_Personas[i].astype(per_datatype[i])

#History_Application表
his_app_var_attribute, his_app_var_numeric, his_app_datatype = get_data_type('History_Application')
train_History_Application.loc[:,his_app_var_numeric] = train_History_Application.loc[:,his_app_var_numeric].fillna(0)
train_History_Application.loc[:,his_app_var_attribute] = train_History_Application.loc[:,his_app_var_attribute].fillna(-1)
for i in col_his_app:
    train_History_Application[i] = train_History_Application[i].astype(his_app_datatype[i])

A_History_Application.loc[:,his_app_var_numeric] = A_History_Application.loc[:,his_app_var_numeric].fillna(0)
A_History_Application.loc[:,his_app_var_attribute] = A_History_Application.loc[:,his_app_var_attribute].fillna(-1)
for i in col_his_app:
    A_History_Application[i] = A_History_Application[i].astype(his_app_datatype[i])
    
    
#History_Payment表
his_pay_var_attribute, his_pay_var_numeric, his_pay_datatype = get_data_type('History_Payment')
train_History_Payment.loc[:,his_pay_var_numeric] = train_History_Payment.loc[:,his_pay_var_numeric].fillna(0)
train_History_Payment.loc[:,his_pay_var_attribute] = train_History_Payment.loc[:,his_pay_var_attribute].fillna(-1)
for i in col_his_pay:
    train_History_Payment[i] = train_History_Payment[i].astype(his_pay_datatype[i])
    
his_pay_var_attribute, his_pay_var_numeric, his_pay_datatype = get_data_type('History_Payment')
A_History_Payment.loc[:,his_pay_var_numeric] = A_History_Payment.loc[:,his_pay_var_numeric].fillna(0)
A_History_Payment.loc[:,his_pay_var_attribute] = A_History_Payment.loc[:,his_pay_var_attribute].fillna(-1)
for i in col_his_pay:
    A_History_Payment[i] = A_History_Payment[i].astype(his_pay_datatype[i])



In [0]:
# 类型变量列表
var_attribute=app_var_attribute+per_var_attribute+his_app_var_attribute+his_pay_var_attribute
#数值变量列表
var_numeric=app_var_numeric+per_var_numeric+his_app_var_numeric+his_pay_var_numeric

In [15]:
print(A_History_Application.columns)
print(A_Application.columns)

Index(['历史申请编号', '申请编号', '贷款类型', '贷款年金', '申请额度', '信用额度', '首付金额', '商品价格',
       '申请周内日', '申请时点', '是否为最后一次申请记录', '是否为当天最后一次申请记录', '首付率', '利率1', '利率2',
       '贷款用途', '合同状态', '历史贷款授信距本次申请时间', '付款方式', '拒绝原因', '陪同申请人', '客户类型',
       '商品类别', '组合类型', '产品类型', '获客渠道', '销售区域', '行业', '贷款期数', '收益级别', '产品组合',
       '首付时间'],
      dtype='object')
Index(['申请编号', '贷款类型', '信用额度', '贷款年金', '商品价格', '陪同申请人', '出生日期距申请日期天数',
       '工作日期距申请日期天数', '注册日期距申请日期天数', '身份认证日期距申请日期天数', '是否提供手机号', '是否提供电话',
       '手机号是否有效', '是否提供email', '申请周内日', '申请时点', '外部评分', '申请人是否额外提供了文件2',
       '申请人是否额外提供了文件3', '申请人是否额外提供了文件4', '申请人是否额外提供了文件5', '申请人是否额外提供了文件6',
       '申请人是否额外提供了文件7', '申请人是否额外提供了文件8', '申请人是否额外提供了文件9', '贷款申请前1小时内征信查询次数',
       '贷款申请前1天内征信查询次数', '贷款申请前1周内征信查询次数', '贷款申请前1个月内征信查询次数',
       '贷款申请前1个季度内征信查询次数', '贷款申请前1年内征信查询次数'],
      dtype='object')


# 特征处理

## History_Payment

### 特征添加

In [0]:
#train_History_Payment
def cal(x):
    return pd.Series([len(x),  #还款笔数
        x['历史申请编号'].nunique(),
        x["分期付款日期"].mean(),
        x["有支付行为的分期付款"].mean(),
        x['分期付款应支付时间'].mean(),
        x['分期付款实际支付时间'].mean(),
        x['本期贷款金额'].mean(),
        x['本期还款金额'].mean()
        ])
#训练集数据清洗
pay=train_History_Payment.groupby('申请编号').apply(lambda x : cal(x))
pay_mean=pay
pay_mean.columns=['还款笔数','历史申请次数', '平均分期付款日期', '平均有支付行为的分期付款', '平均分期付款应支付时间','平均分期付款实际支付时间','平均本期贷款金额','平均本期还款金额']
pay_mean['申请编号']=pay_mean.index
pay_mean=pay_mean.reset_index(drop=True)

#应用集数据清洗
A_pay=A_History_Payment.groupby('申请编号').apply(lambda x : cal(x))
A_pay_mean=A_pay
A_pay_mean.columns=['还款笔数','历史申请次数', '平均分期付款日期', '平均有支付行为的分期付款', '平均分期付款应支付时间','平均分期付款实际支付时间','平均本期贷款金额','平均本期还款金额']
A_pay_mean['申请编号']=A_pay_mean.index
A_pay_mean=A_pay_mean.reset_index(drop=True)

#标签集
col_hist_pay_new = ['还款笔数','历史申请次数', '平均分期付款日期', '平均有支付行为的分期付款', '平均分期付款应支付时间','平均分期付款实际支付时间','平均本期贷款金额','平均本期还款金额']
his_pay_var_attribute_new = []
his_pay_var_numeric_new = ['还款笔数','历史申请次数', '平均分期付款日期', '平均有支付行为的分期付款', '平均分期付款应支付时间','平均分期付款实际支付时间','平均本期贷款金额','平均本期还款金额']


# ----------------------------------------------------------数据合并
#1.训练集
pay_mean.loc[:,his_pay_var_numeric_new] = pay_mean.loc[:,his_pay_var_numeric_new].fillna(0)
pay_mean.loc[:,his_pay_var_attribute_new] = pay_mean.loc[:,his_pay_var_attribute_new].fillna(-1)
#2. 测试集
A_pay_mean.loc[:,his_pay_var_numeric_new] = A_pay_mean.loc[:,his_pay_var_numeric_new].fillna(0)
A_pay_mean.loc[:,his_pay_var_attribute_new] = A_pay_mean.loc[:,his_pay_var_attribute_new].fillna(-1)
# data=pd.merge(train_Application,train_Personas,on="申请编号",how="outer")
# data=pd.merge(data,pay_mean,on="申请编号",how="left")
# data = pd.merge(data,train_label,on="申请编号",how="left")
# #data=pd.merge(data,app,on="申请编号",how="outer")
# data.index = data['申请编号']
# data.drop('申请编号', axis=1, inplace=True)
# data.loc[:,his_pay_var_numeric_new] = data.loc[:,his_pay_var_numeric_new].fillna(0)
# data.loc[:,his_pay_var_attribute_new] = data.loc[:,his_pay_var_attribute_new].fillna(-1)


In [0]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

### 缺失值替换、类别变量编码

In [0]:
#1.训练集
pay_mean.loc[:,his_pay_var_numeric_new] = pay_mean.loc[:,his_pay_var_numeric_new].fillna(0)
pay_mean.loc[:,his_pay_var_attribute_new] = pay_mean.loc[:,his_pay_var_attribute_new].fillna(-1)
#2. 测试集
A_pay_mean.loc[:,his_pay_var_numeric_new] = A_pay_mean.loc[:,his_pay_var_numeric_new].fillna(0)
A_pay_mean.loc[:,his_pay_var_attribute_new] = A_pay_mean.loc[:,his_pay_var_attribute_new].fillna(-1)

data['是否为最后一次申请记录']=data['是否为最后一次申请记录'].map({'Y':1,'N':0,np.nan:-1})
data['贷款类型']=data['贷款类型'].map({1:1,0:0,np.nan:-1,'A':2,-1:-1})

In [0]:
#train_History_Application
#数值型变量求平均
tha_ng=train_History_Application.loc[:,['申请编号']+list(his_app_var_numeric)].groupby('申请编号')
tha_nm=tha_ng.apply(lambda x:x.mean())
#类别型变量求最多的类别
tha_ag=train_History_Application.loc[:,his_app_var_attribute].groupby('申请编号')
tha_am=tha_ag.apply(lambda x:x.apply(lambda x:x.mode()[0]))

C:\Users\--\AppData\Roaming\Python\Python37\site-packages\pandas\core\algorithms.py:829: UserWarning: Unable to sort modes: '<' not supported between instances of 'str' and 'int'
  warn("Unable to sort modes: {error}".format(error=e))
C:\Users\--\AppData\Roaming\Python\Python37\site-packages\pandas\core\algorithms.py:829: UserWarning: Unable to sort modes: '<' not supported between instances of 'int' and 'str'
  warn("Unable to sort modes: {error}".format(error=e))


In [0]:
tha_nm=tha_nm.reset_index()
tha_am=tha_am.reset_index(drop=True)

In [0]:
#A_History_Application
#数值型变量求平均
aha_ng=A_History_Application.loc[:,['申请编号']+list(his_app_var_numeric)].groupby('申请编号')
aha_nm=aha_ng.apply(lambda x:x.mean())
#类别型变量求最多的类别
aha_ag=A_History_Application.loc[:,his_app_var_attribute].groupby('申请编号')
aha_am=aha_ag.apply(lambda x:x.apply(lambda x:x.mode()[0]))

In [0]:
aha_nm=aha_nm.reset_index(drop=True)
aha_am=aha_am.reset_index(drop=True)

In [0]:
# joblib.dump(data,'./data_all.pkl')
# joblib.dump(test,'./test_all.pkl')

['./test_all.pkl']

In [0]:
from sklearn.externals import joblib
test=joblib.load('./test_all.pkl')
data=joblib.load('./data_all.pkl')

In [0]:
# data=pd.merge(data,pay_mean,on="申请编号",how="left")
# data=pd.merge(tha_am,tha_nm,on="申请编号",how="outer")
# data=pd.merge(train_Application,data,on="申请编号",how="left")
# data = pd.merge(data,train_Personas,on="申请编号",how="left")
# data=pd.merge(data,train_label,on="申请编号",how="outer")

In [0]:
# test=pd.merge(aha_am,aha_nm,on="申请编号",how="outer")
# test=pd.merge(A_Application,test,on="申请编号",how="left")
# test = pd.merge(test,A_pay_mean,on="申请编号",how="left")
# test = pd.merge(test,A_Personas,on="申请编号",how="left")

In [0]:
data

,申请编号,贷款类型_x,信用额度_x,贷款年金_x,商品价格_x,陪同申请人_x,出生日期距申请日期天数,工作日期距申请日期天数,注册日期距申请日期天数,身份认证日期距申请日期天数,是否提供手机号,是否提供电话,手机号是否有效,是否提供email,申请周内日_x,申请时点_x,外部评分,申请人是否额外提供了文件2,申请人是否额外提供了文件3,申请人是否额外提供了文件4,申请人是否额外提供了文件5,申请人是否额外提供了文件6,申请人是否额外提供了文件7,申请人是否额外提供了文件8,申请人是否额外提供了文件9,贷款申请前1小时内征信查询次数,贷款申请前1天内征信查询次数,贷款申请前1周内征信查询次数,贷款申请前1个月内征信查询次数,贷款申请前1个季度内征信查询次数,贷款申请前1年内征信查询次数,历史申请编号,贷款类型_y,是否为最后一次申请记录,是否为当天最后一次申请记录,贷款用途,合同状态,付款方式,拒绝原因,客户类型,商品类别,组合类型,产品类型,获客渠道,销售区域,行业,收益级别,产品组合,贷款年金_y,申请额度,信用额度_y,首付金额,商品价格_y,申请周内日_y,申请时点_y,首付率,利率1,利率2,历史贷款授信距本次申请时间,陪同申请人_y,贷款期数,首付时间,性别,是否有车,是否有房,孩子个数,客户收入,收入类型,教育程度,婚姻状态,居住状态,居住地人口密集度,有车时间,职业,家庭成员数,居住信息1,居住信息2,居住信息3,居住信息4,客户居住地评分1,客户居住地评分2,地址是否一致标志1,地址是否一致标志2,地址是否一致标志3,地址是否一致标志4,地址是否一致标志5,地址是否一致标志6,单位类型,社交圈违约信息2_2,社交圈违约信息2_1,社交圈违约信息1_2,社交圈违约信息1_1,最近一次换手机号码距申请日天数,标签,还款笔数,历史申请次数,平均分期付款日期,平均有支付行为的分期付款,平均分期付款应支付时间,平均分期付款实际支付时间,平均本期贷款金额,平均本期还款金额
0,0,0,460190,17463,419951,7,-10107,-342,-5421,-3292,1,1,1,0,2,9,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,779169.0,A,Y,1.0,23.0,0.0,0.0,7.0,0.0,19.0,3.0,0.0,7.0,10.0,2.0,1.0,14.0,1100.000000,9507.000000,9403.000000,960.000000,9507.000000,6.000000,9.000000,0.100002,0.000000,0.000000,-1774.000000,0.000000,14.000000,292204.000000,1,0,0,0,237349,1,2,3,1,0.022330,0,6,1,0.596306,0.076194,0.115705,0.110490,0,0,0,0,0,0,0,0,5,0,0,0,0,-887,1,15.0,1.0,2.000000,9.066667,-1600.000000,-1610.333333,1098.133333,880.266667
1,1,0,424370,15585,384597,7,-13980,-1110,-3387,-826,1,0,1,0,2,7,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,4,313225.0,A,Y,1.0,23.0,0.0,0.0,7.0,0.0,19.0,3.0,0.0,4.0,46.0,2.0,1.0,14.0,811.000000,4992.000000,6360.000000,11.000000,4992.000000,4.000000,10.000000,0.000000,0.000000,0.000000,-425.500000,0.000000,14.000000,292204.000000,1,0,1,0,237236,7,4,1,1,0.018751,0,9,2,0.000000,0.000000,0.000000,0.000000,1,1,0,1,0,0,0,1,5,0,0,0,0,-271,0,25.0,2.0,2.000000,8.680000,-267.920000,-272.480000,803.520000,778.520000
2,2,0,469330,18837,419951,1,-13331,-2246,-3870,-171,1,0,1,0,1,7,0.503654,0,0,0,0,0,0,0,0,0,0,0,0,0,4,210470.0,0,Y,1.0,23.0,2.0,3.0,7.0,2.0,27.0,2.0,1.0,4.0,15.0,2.0,0.0,3.0,2952.300000,39215.000000,40573.900000,4.400000,39213.900000,6.100000,7.200000,0.000000,0.000000,0.000000,-409.600000,0.200000,12.200000,87624.800000,0,1,1,-1,237321,1,4,1,1,0.025895,1,15,1,0.544952,0.076318,0.115296,0.111989,0,0,1,0,1,0,0,0,5,0,0,0,0,-332,0,41.0,4.0,1.975610,8.975610,-331.829268,-334.439024,5578.146341,5455.414634
3,3,0,464188,16754,409850,7,-16540,292204,-970,-2916,0,0,1,0,5,5,0.000000,0,0,0,0,0,0,0,1,0,0,0,0,0,1,889570.0,A,Y,1.0,23.0,0.0,0.0,7.0,0.0,6.0,3.0,0.0,7.0,77.0,3.0,4.0,12.0,4912.000000,51095.000000,49020.000000,5119.000000,51095.000000,7.000000,5.000000,0.102803,0.000000,0.000000,-409.000000,0.000000,14.000000,292204.000000,0,0,1,-1,237304,3,4,2,1,0.018817,0,-1,0,0.000000,0.000000,0.000000,0.000000,1,1,0,0,0,0,0,0,57,1,0,1,0,-204,0,12.0,1.0,2.000000,8.416667,-252.000000,-267.000000,4911.500000,4911.500000
4,5,0,434196,16165,394698,7,-17919,-11037,-9350,-3588,1,0,1,0,0,6,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,5,357652.0,0,Y,1.0,23.0,0.0,0.0,7.0,2.0,27.0,2.0,0.0,5.0,0.0,10.0,0.0,1.0,2414.600000,24482.200000,25609.200000,81.400000,24480.000000,3.400000,7.200000,0.023563,0.000000,0.000000,-645.600000,1.400000,8.000000,116816.800000,0,0,0,-1,237321,1,1,2,1,0.025895,0,10,0,0.000000,0.000000,0.115851,0.111411,0,0,1,0,1,1,0,1,53,0,0,0,0,-1049,0,27.0,4.0,1.592593,6.185185,-1107.925926,-1111.407407,1957.629630,1923.222222
5,8,0,472377,16501,419951,7,-9217,-265,-4264,-2746,1,0,1,0,0,5,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,1,356284.0,0,Y,1.0,23.0,0.0,0.0,7.0,2.0,27.0,2.0,0.0,4.0,0.0,10.0,1.0,6.0,4390.000000,62320.750000,67916.250000,651.750000,62320.750000,2.750000,5.000000,0.052255,0.000000,0.000000,-465.000000,2.000000,23.750000,219153.000000,0,1,0,0,237519,7,4,1,1,0.019238,1,3,2,0.000000,0.076696,0.122013,0.115973,2,0,0,0,0,0,0,0,51,0,0,0,0,-357,0,50.0,3.0,2.000000,13.460000,-291.680000,-300.280000,7532.620000,7107.420000
6,9,0,424045,15783,386113,7,-8174,-758,-2749,-1294,1,1,1,0,5,8,0.251527,0,0,0,0,0,0,0,0,0,0,0,0

In [0]:
data['贷款类型_y']= data['贷款类型_y'].fillna(-1)

In [0]:
data['贷款类型_y'].unique()

array(['A', '0', nan, '1', -1], dtype=object)

In [0]:
data['是否为最后一次申请记录'].unique()

array(['Y', nan, 'N'], dtype=object)

In [0]:
data['是否为最后一次申请记录']=data['是否为最后一次申请记录'].map({'Y':1,'N':0,np.nan:-1})
data['贷款类型_y']=data['贷款类型_y'].map({1:1,0:0,np.nan:-1,'A':2,-1:-1})

In [0]:
data=data.fillna(-1)

In [0]:
train=data.drop(['标签','申请编号'],axis=1)
train_best=train.values
test_best=test.drop('申请编号',axis=1).values
y_train=data['标签'].values

In [0]:
train.shape

(140000, 100)

In [0]:
test.shape

(21511, 101)

In [0]:
def feval_ks(preds, train_data):
    from sklearn.metrics import roc_curve
    fpr, tpr, threshold = roc_curve( train_data.get_label(),preds)
    ks = max(tpr-fpr)
    return 'KS',ks,True

In [0]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import warnings
import time
import sys
import os
import re
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',None)
pd.set_option('max_colwidth',100)

# lgb
param = {'num_leaves': 120,
         'min_data_in_leaf': 30, 
         'objective':'binary',
         'max_depth': 10,
         'learning_rate': 0.01,
         "min_child_samples": 30,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         # "metric": 'auc',
         "lambda_l1": 0.1,
         "verbosity": -1}
folds = KFold(n_splits=5, shuffle=True, random_state=2018)
oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_best, y_train)):
    print("fold n°{}".format(fold_+1))
    trn_data = lgb.Dataset(train_best[trn_idx], y_train[trn_idx])
    val_data = lgb.Dataset(train_best[val_idx], y_train[val_idx])

    num_round = 10
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=200, feval=feval_ks,early_stopping_rounds = 100)
    oof_lgb[val_idx] = clf.predict(train_best[val_idx], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test_best, num_iteration=clf.best_iteration) / folds.n_splits

# print(feval_ks(oof_lgb,y_train))

fold n°1
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's binary_logloss: 0.416095	training's KS: 0.339768	valid_1's binary_logloss: 0.419353	valid_1's KS: 0.286643
fold n°2
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's binary_logloss: 0.417615	training's KS: 0.344228	valid_1's binary_logloss: 0.413115	valid_1's KS: 0.282246
fold n°3
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's binary_logloss: 0.415654	training's KS: 0.347124	valid_1's binary_logloss: 0.420953	valid_1's KS: 0.275664
fold n°4
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[10]	training's binary_logloss: 0.41532	training's KS: 0.349227	valid_1's binary_logloss: 0.42292	valid_1's KS: 0.282363
fold n°5
Training until validation

In [0]:
y_train